<a href="https://colab.research.google.com/github/HelmiAlf/Projects/blob/main/%5BNLP%5D_Word_Sense_Disambiguation_(WSD)_bahasa_indonesia.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Modules

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/My Drive/Colab Notebooks

Mounted at /content/drive
/content/drive/My Drive/Colab Notebooks


In [ ]:
!pip install PySastrawi
!git clone https://github.com/bahasa-csui/aksara
!apt-get install foma-bin
!pip install -r aksara/requirements.txt

     |████████████████████████████████| 215kB 3.1MB/s 
fatal: destination path 'aksara' already exists and is not an empty directory.
Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libfoma0
The following NEW packages will be installed:
  foma-bin libfoma0
0 upgraded, 2 newly installed, 0 to remove and 34 not upgraded.
Need to get 286 kB of archives.
After this operation, 1,041 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libfoma0 amd64 0.9.18+r243-1build1 [104 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 foma-bin amd64 0.9.18+r243-1build1 [182 kB]
Fetched 286 kB in 2s (169 kB/s)
Selecting previously unselected package libfoma0:amd64.
(Reading database ... 160706 files and di

In [ ]:
import math
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report 
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

from nltk.collocations import BigramCollocationFinder
from nltk.metrics import BigramAssocMeasures

%matplotlib inline

# II. Word Sense Disambiguation (WSD)

Fitur2 yang dicoba : 
*  Cooccurence
*  Co-occurrences with removed stopwords and stemmed words 
*  Collocation
*  Co-occurrences with top 25% words
* Part-of-speech tag cooccurrences

note: dataset sudah difilter

In [ ]:
def evaluate(y_test, y_pred): 
    print("* Confusion Matrix") 
    print(pd.DataFrame(confusion_matrix(y_test, y_pred), \
            index = ['Actual Class ' + str(i) for i in range(len(np.unique(y_test)))], \
            columns = ['Pred Class ' + str(i) for i in range(len(np.unique(y_test)))] )) 
      
    print ("\n* Accuracy : ", 
    accuracy_score(y_test,y_pred)*100) 
      
    print("\n* Classification Report :\n", 
    classification_report(y_test, y_pred, zero_division=0)) 

In [ ]:
stopword = StopWordRemoverFactory().create_stop_word_remover()
stemmer = StemmerFactory().create_stemmer()

def clean_sentence(sentence):
    sentence = stopword.remove(sentence)
    sentence = stemmer.stem(sentence)
    
    return sentence

In [ ]:
def clean(text):
    text = text.lower()
    text = stopword.remove(text)
    text = stemmer.stem(text)
    return text.split()

def get_not_target(target, tupl):
    if tupl[0] != target:
        return i[0]
    return tupl[1]

def get_collocation_data(source_df, target_df):
    for i in range(source_df.shape[0]):
        row = source_df.loc[i]
        words = clean(row["kalimat"])
        finder = BigramCollocationFinder.from_words(words)
        finder.apply_ngram_filter(filter_ngram)
    
        new_row = []
        for i in collocated_tuple:
            score = finder.score_ngram(bigram_measures.raw_freq, i[0], i[1])
            if score is None:
                score = 0
            else:
                score = score * len(words)
            new_row.append(score)
        series = pd.Series(new_row, index = target_df.columns)
        target_df = target_df.append(series, ignore_index=True)
    return target_df

In [ ]:
def get_pos_tags(kalimat, target):
  result = ""

  with open('input.txt', 'w') as input_file:
    input_file.write(kalimat)
    input_file.close()


  !python3 main.py -f 'input.txt' --output "output_example.conllu" 
  

  with open('output_example.conllu', 'r') as f:
    while True:
      line = f.readline()
      if not line:
        break
      splitted = line.split("\t")
      if len(splitted) <=1:
        continue
      else:
        if target == splitted[1]:
          result += target+" "
        else:
          result += splitted[3]+" "
  return result

## 2 senses words

In [ ]:
target = 'lebat'

### Co-occurrences

In [ ]:
train_data = pd.read_csv("train_2_sense.csv")
test_data = pd.read_csv("test_2_sense.csv")

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_data["kalimat"]).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_data["kalimat"]).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             0             7

* Accuracy :  100.0

* Classification Report :
               precision    recall  f1-score   support

        5501       1.00      1.00      1.00        16
        5502       1.00      1.00      1.00         7

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             4             3

* Accuracy :  82.6086956521739

* Classification Report :
               precision    recall  f1-score   support

        5501       0.80      1.00      0.89        16
        5502       1.00      0.43      0.60         7

    accuracy                           0.83        23
   macro avg       0.90      0.71      0.74        23
weighted avg       0.86      0.83      0.80        23



### Co-occurrences with removed stopwords and stemmed words

In [ ]:
train_data = pd.read_csv("train_2_sense.csv")
test_data = pd.read_csv("test_2_sense.csv")

In [ ]:
train_kalimat = train_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)
test_kalimat = test_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_kalimat).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_kalimat).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             0             7

* Accuracy :  100.0

* Classification Report :
               precision    recall  f1-score   support

        5501       1.00      1.00      1.00        16
        5502       1.00      1.00      1.00         7

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             2             5

* Accuracy :  91.30434782608695

* Classification Report :
               precision    recall  f1-score   support

        5501       0.89      1.00      0.94        16
        5502       1.00      0.71      0.83         7

    accuracy                           0.91        23
   macro avg       0.94      0.86      0.89        23
weighted avg       0.92      0.91      0.91        23



### Collocation

In [ ]:
train_data = pd.read_csv("train_2_sense.csv")
test_data = pd.read_csv("test_2_sense.csv")

In [ ]:
text = train_data['kalimat'].str.cat(sep=' ')
text = clean(text)

bigram_measures = BigramAssocMeasures()

filter_ngram = lambda *w: target not in w

finder = BigramCollocationFinder.from_words(text, window_size=3)
finder.apply_freq_filter(2)
finder.apply_ngram_filter(filter_ngram)
collocated_tuple = finder.nbest(bigram_measures.raw_freq, 200)
collocated_tuple.sort()

In [ ]:
collocated = []
for i in collocated_tuple:
    collocated.append(get_not_target(target, i))

collocation_train_data = pd.DataFrame(columns=collocated)
collocation_test_data = pd.DataFrame(columns=collocated)

collocation_train_data = get_collocation_data(train_data, collocation_train_data)
collocation_test_data = get_collocation_data(test_data, collocation_test_data)

In [ ]:
X_train = collocation_train_data
y_train = train_data["sense"]
X_test = collocation_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             1             6

* Accuracy :  95.65217391304348

* Classification Report :
               precision    recall  f1-score   support

        5501       0.94      1.00      0.97        16
        5502       1.00      0.86      0.92         7

    accuracy                           0.96        23
   macro avg       0.97      0.93      0.95        23
weighted avg       0.96      0.96      0.96        23



In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             2             5

* Accuracy :  91.30434782608695

* Classification Report :
               precision    recall  f1-score   support

        5501       0.89      1.00      0.94        16
        5502       1.00      0.71      0.83         7

    accuracy                           0.91        23
   macro avg       0.94      0.86      0.89        23
weighted avg       0.92      0.91      0.91        23



### Co-occurrences with top 25% words

In [ ]:
train_data = pd.read_csv("train_2_sense.csv")
test_data = pd.read_csv("test_2_sense.csv")

In [ ]:
train_kalimat = train_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)
test_kalimat = test_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_kalimat).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_kalimat).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
boundary = clean_train_vector.sum().quantile(0.75)
column_to_drop = [col for col, val in clean_train_vector.sum().iteritems() if val < boundary]

clean_train_vector = clean_train_vector.drop(columns=column_to_drop)
clean_test_data = clean_test_data.drop(columns=column_to_drop)

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             0             7

* Accuracy :  100.0

* Classification Report :
               precision    recall  f1-score   support

        5501       1.00      1.00      1.00        16
        5502       1.00      1.00      1.00         7

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            16             0
Actual Class 1             0             7

* Accuracy :  100.0

* Classification Report :
               precision    recall  f1-score   support

        5501       1.00      1.00      1.00        16
        5502       1.00      1.00      1.00         7

    accuracy                           1.00        23
   macro avg       1.00      1.00      1.00        23
weighted avg       1.00      1.00      1.00        23



### Part-of-speech tag + Cooccurrences

In [ ]:
train_data = pd.read_csv("train_2_sense.csv")
test_data = pd.read_csv("test_2_sense.csv")

In [ ]:
%cd aksara
train_data["POS"] = train_data.apply(lambda row: get_pos_tags(row["kalimat"], target), axis=1) 
test_data["POS"] = train_data.apply(lambda row: get_pos_tags(row["kalimat"], target), axis=1) 
%cd ..

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_data["POS"]).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_data["POS"]).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            10             6
Actual Class 1             5             2

* Accuracy :  52.17391304347826

* Classification Report :
               precision    recall  f1-score   support

        5501       0.67      0.62      0.65        16
        5502       0.25      0.29      0.27         7

    accuracy                           0.52        23
   macro avg       0.46      0.46      0.46        23
weighted avg       0.54      0.52      0.53        23



In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1
Actual Class 0            13             3
Actual Class 1             6             1

* Accuracy :  60.86956521739131

* Classification Report :
               precision    recall  f1-score   support

        5501       0.68      0.81      0.74        16
        5502       0.25      0.14      0.18         7

    accuracy                           0.61        23
   macro avg       0.47      0.48      0.46        23
weighted avg       0.55      0.61      0.57        23



### Comparison & Conclusion

Berikut ringkasan evaluasi model:


| Feature      | Model&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; | Evaluation &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;|
| ----------- | ----------- | ----------- |
| Cooccurrence      | MultinomialNB       | Accuracy : 100% |
|    |         | Precision : 100%|
|    |         | Recall : 100%|
|    |         | F1-score : 100% |
|    | BernoulliNB       | Accuracy : 83%|
|    |         | Precision : 90%|
|    |         | Recall : 71%|
|    |         | F1-score : 74%|
| Cooccurrence & data cleaning      | MultinomialNB       | Accuracy : 100%|
|    |         | Precision : 100%|
|    |         | Recall : 100%|
|    |         | F1-score : 100%|
|    | BernoulliNB       | Accuracy : 91%|
|    |         | Precision : 94%|
|    |         | Recall : 86%|
|    |         | F1-score : 89%|
| Collocation      | MultinomialNB       | Accuracy : 96%|
|    |         | Precision : 97%|
|    |         | Recall : 93%|
|    |         | F1-score : 95%|
|    | BernoulliNB       | Accuracy : 91%|
|    |         | Precision : 94%|
|    |         | Recall : 86%|
|    |         | F1-score : 89%|
| Cooccurrence top 25%:      | MultinomialNB       | Accuracy : 100%|
|    |         | Precision :100% |
|    |         | Recall : 100%|
|    |         | F1-score : 100%|
|    | BernoulliNB       | Accuracy : 100%|
|    |         | Precision :100% |
|    |         | Recall : 100%|
|    |         | F1-score : 100%|
| POS tag Cooccurrence:      | MultinomialNB       | Accuracy : 52.1%|
|    |         | Precision : 46%|
|    |         | Recall : 46%|
|    |         | F1-score : 46%|
|    | BernoulliNB       | Accuracy :  61%|
|    |         | Precision : 47%|
|    |         | Recall : 48%|
|    |         | F1-score : 46%|


Dapat ditarik kesimpulan bahwa fitur cooccurence dengan top 25% words adalah model terbaik dan fitur POS tag adalah fitur terburuk


Terlihat bahwa fitur cooccurence sudah cukup baik tanpa dilakukannya data cleaning, namun performa model meningkat jika dilakukan.



## 6 senses words

In [ ]:
target = 'berat'

### Co-occurrences

In [ ]:
train_data = pd.read_csv("train_6_sense.csv")
test_data = pd.read_csv("test_6_sense.csv")

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_data["kalimat"]).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_data["kalimat"]).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            27            12  ...             0             0
Actual Class 1             4             8  ...             0             0
Actual Class 2             7             3  ...             0             0
Actual Class 3             6             3  ...             1             2
Actual Class 4             2             6  ...             0             1

[5 rows x 5 columns]

* Accuracy :  45.23809523809524

* Classification Report :
               precision    recall  f1-score   support

        5201       0.59      0.68      0.63        40
        5202       0.25      0.67      0.36        12
        5203       0.50      0.09      0.15        11
        5204       1.00      0.08      0.15        12
        5206       0.33      0.11      0.17         9

    accuracy                           0.45        84
   macro avg       0.53      0.33      0.29        84
weighted

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            40             0  ...             0             0
Actual Class 1            12             0  ...             0             0
Actual Class 2            11             0  ...             0             0
Actual Class 3            12             0  ...             0             0
Actual Class 4             9             0  ...             0             0

[5 rows x 5 columns]

* Accuracy :  47.61904761904761

* Classification Report :
               precision    recall  f1-score   support

        5201       0.48      1.00      0.65        40
        5202       0.00      0.00      0.00        12
        5203       0.00      0.00      0.00        11
        5204       0.00      0.00      0.00        12
        5206       0.00      0.00      0.00         9

    accuracy                           0.48        84
   macro avg       0.10      0.20      0.13        84
weighted

### Co-occurrences with removed stopwords and stemmed words

In [ ]:
train_data = pd.read_csv("train_6_sense.csv")
test_data = pd.read_csv("test_6_sense.csv")

In [ ]:
train_kalimat = train_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)
test_kalimat = test_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_kalimat).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_kalimat).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            30             9  ...             1             0
Actual Class 1             7             4  ...             0             1
Actual Class 2             6             2  ...             0             0
Actual Class 3             8             1  ...             2             1
Actual Class 4             4             1  ...             0             1

[5 rows x 5 columns]

* Accuracy :  47.61904761904761

* Classification Report :
               precision    recall  f1-score   support

        5201       0.55      0.75      0.63        40
        5202       0.24      0.33      0.28        12
        5203       0.50      0.27      0.35        11
        5204       0.67      0.17      0.27        12
        5206       0.33      0.11      0.17         9

    accuracy                           0.48        84
   macro avg       0.46      0.33      0.34        84
weighted

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            40             0  ...             0             0
Actual Class 1            12             0  ...             0             0
Actual Class 2            11             0  ...             0             0
Actual Class 3            12             0  ...             0             0
Actual Class 4             9             0  ...             0             0

[5 rows x 5 columns]

* Accuracy :  47.61904761904761

* Classification Report :
               precision    recall  f1-score   support

        5201       0.48      1.00      0.65        40
        5202       0.00      0.00      0.00        12
        5203       0.00      0.00      0.00        11
        5204       0.00      0.00      0.00        12
        5206       0.00      0.00      0.00         9

    accuracy                           0.48        84
   macro avg       0.10      0.20      0.13        84
weighted

### Collocation

In [ ]:
train_data = pd.read_csv("train_6_sense.csv")
test_data = pd.read_csv("test_6_sense.csv")

In [ ]:
text = train_data['kalimat'].str.cat(sep=' ')
text = clean(text)

bigram_measures = BigramAssocMeasures()

filter_ngram = lambda *w: target not in w

finder = BigramCollocationFinder.from_words(text, window_size=3)
finder.apply_freq_filter(2)
finder.apply_ngram_filter(filter_ngram)
collocated_tuple = finder.nbest(bigram_measures.raw_freq, 200)
collocated_tuple.sort()

In [ ]:
collocated = []
for i in collocated_tuple:
    collocated.append(get_not_target(target, i))

collocation_train_data = pd.DataFrame(columns=collocated)
collocation_test_data = pd.DataFrame(columns=collocated)

collocation_train_data = get_collocation_data(train_data, collocation_train_data)
collocation_test_data = get_collocation_data(test_data, collocation_test_data)

In [ ]:
X_train = collocation_train_data
y_train = train_data["sense"]
X_test = collocation_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            40             0  ...             0             0
Actual Class 1            10             1  ...             0             1
Actual Class 2             8             0  ...             0             0
Actual Class 3            12             0  ...             0             0
Actual Class 4             5             0  ...             0             0

[5 rows x 5 columns]

* Accuracy :  52.38095238095239

* Classification Report :
               precision    recall  f1-score   support

        5201       0.53      1.00      0.70        40
        5202       1.00      0.08      0.15        12
        5203       0.43      0.27      0.33        11
        5204       0.00      0.00      0.00        12
        5206       0.00      0.00      0.00         9

    accuracy                           0.52        84
   macro avg       0.39      0.27      0.24        84
weighted

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            40             0  ...             0             0
Actual Class 1            10             1  ...             0             1
Actual Class 2            11             0  ...             0             0
Actual Class 3            12             0  ...             0             0
Actual Class 4             5             0  ...             0             4

[5 rows x 5 columns]

* Accuracy :  53.57142857142857

* Classification Report :
               precision    recall  f1-score   support

        5201       0.51      1.00      0.68        40
        5202       1.00      0.08      0.15        12
        5203       0.00      0.00      0.00        11
        5204       0.00      0.00      0.00        12
        5206       0.80      0.44      0.57         9

    accuracy                           0.54        84
   macro avg       0.46      0.31      0.28        84
weighted

### Co-occurrences with top 25% words

In [ ]:
train_data = pd.read_csv("train_6_sense.csv")
test_data = pd.read_csv("test_6_sense.csv")

In [ ]:
train_kalimat = train_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)
test_kalimat = test_data.apply(lambda row: clean_sentence(row['kalimat']), axis=1)

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_kalimat).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_kalimat).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
boundary = clean_train_vector.sum().quantile(0.75)
column_to_drop = [col for col, val in clean_train_vector.sum().iteritems() if val < boundary]

clean_train_vector = clean_train_vector.drop(columns=column_to_drop)
clean_test_data = clean_test_data.drop(columns=column_to_drop)

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            32             5  ...             1             1
Actual Class 1             8             2  ...             0             2
Actual Class 2             7             1  ...             0             0
Actual Class 3            11             0  ...             1             0
Actual Class 4             6             1  ...             0             1

[5 rows x 5 columns]

* Accuracy :  46.42857142857143

* Classification Report :
               precision    recall  f1-score   support

        5201       0.50      0.80      0.62        40
        5202       0.22      0.17      0.19        12
        5203       0.60      0.27      0.37        11
        5204       0.50      0.08      0.14        12
        5206       0.25      0.11      0.15         9

    accuracy                           0.46        84
   macro avg       0.41      0.29      0.30        84
weighted

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            40             0  ...             0             0
Actual Class 1            12             0  ...             0             0
Actual Class 2            11             0  ...             0             0
Actual Class 3            12             0  ...             0             0
Actual Class 4             9             0  ...             0             0

[5 rows x 5 columns]

* Accuracy :  47.61904761904761

* Classification Report :
               precision    recall  f1-score   support

        5201       0.48      1.00      0.65        40
        5202       0.00      0.00      0.00        12
        5203       0.00      0.00      0.00        11
        5204       0.00      0.00      0.00        12
        5206       0.00      0.00      0.00         9

    accuracy                           0.48        84
   macro avg       0.10      0.20      0.13        84
weighted

### Part-of-speech tag + Cooccurrences

In [ ]:
train_data = pd.read_csv("train_6_sense.csv")
test_data = pd.read_csv("test_6_sense.csv")

In [ ]:
%cd aksara
train_data["POS"] = train_data.apply(lambda row: get_pos_tags(row["kalimat"], target), axis=1) 
test_data["POS"] = train_data.apply(lambda row: get_pos_tags(row["kalimat"], target), axis=1) 
%cd ..

Output streaming akan dipotong hingga 5000 baris terakhir.
['11', 'dengan', 'dengan', 'ADP', '_', '_', '_', '_', '_', '_\n']
['12', 'sangat', 'sangat', 'ADV', '_', '_', '_', '_', '_', '_\n']
['13', 'baik', 'baik', 'ADJ', '_', 'Degree=Pos', '_', '_', '_', 'SpaceAfter=No\n']
['14', ',', ',', 'PUNCT', '_', '_', '_', '_', '_', '_\n']
['15', 'mampu', 'mampu/mampu', 'ADJ/AUX', '_', '(ADJ -> Degree=Pos)', '_', '_', '_', '_\n']
['16', 'bertahan', 'tahan', 'VERB', '_', 'Subcat=Intr|Voice=Act', '_', '_', '_', '_\n']
['17', 'pada', 'pada', 'ADP', '_', '_', '_', '_', '_', '_\n']
['18', 'kerusakan', 'rusak', 'NOUN', '_', 'Number=Sing', '_', '_', '_', '_\n']
['19', 'berat', 'berat', 'ADJ', '_', 'Degree=Pos', '_', '_', '_', '_\n']
['20', 'dan', 'dan', 'CCONJ', '_', '_', '_', '_', '_', '_\n']
['21', 'dapat', 'dapat/dapat', 'AUX/VERB', '_', '_', '_', '_', '_', '_\n']
['22', 'diperbaiki', 'baik', 'VERB', '_', 'Subcat=Tran|Voice=Pass', '_', '_', '_', '_\n']
['23', 'dalam', 'dalam/dalam/dalam', 'NOUN/ADJ/

In [ ]:
cv = CountVectorizer()
train_vector = cv.fit_transform(train_data["POS"]).toarray()
clean_train_vector = pd.DataFrame(train_vector, columns = cv.get_feature_names())


test_vector = cv.transform(test_data["POS"]).toarray()
clean_test_data = pd.DataFrame(test_vector, columns = cv.get_feature_names())

In [ ]:
X_train = clean_train_vector
y_train = train_data["sense"]
X_test = clean_test_data
y_test = test_data["sense"]

In [ ]:
clf = MultinomialNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            17             5  ...             6             9
Actual Class 1             3             1  ...             1             3
Actual Class 2             5             1  ...             2             1
Actual Class 3             2             3  ...             2             3
Actual Class 4             7             0  ...             1             0

[5 rows x 5 columns]

* Accuracy :  26.190476190476193

* Classification Report :
               precision    recall  f1-score   support

        5201       0.50      0.42      0.46        40
        5202       0.10      0.08      0.09        12
        5203       0.17      0.18      0.17        11
        5204       0.17      0.17      0.17        12
        5206       0.00      0.00      0.00         9

    accuracy                           0.26        84
   macro avg       0.19      0.17      0.18        84
weighte

In [ ]:
clf = BernoulliNB()
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)
evaluate(y_test, y_pred)

* Confusion Matrix
                Pred Class 0  Pred Class 1  ...  Pred Class 3  Pred Class 4
Actual Class 0            15            16  ...             2             5
Actual Class 1             4             3  ...             1             2
Actual Class 2             6             5  ...             0             0
Actual Class 3             5             5  ...             1             1
Actual Class 4             6             2  ...             0             0

[5 rows x 5 columns]

* Accuracy :  22.61904761904762

* Classification Report :
               precision    recall  f1-score   support

        5201       0.42      0.38      0.39        40
        5202       0.10      0.25      0.14        12
        5203       0.00      0.00      0.00        11
        5204       0.25      0.08      0.12        12
        5206       0.00      0.00      0.00         9

    accuracy                           0.23        84
   macro avg       0.15      0.14      0.13        84
weighted

### Comparison & Conclusion

### Comparison & Conclusion

Berikut ringkasan evaluasi model:


| Feature      | Model&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp;&nbsp; &nbsp; &nbsp; | Evaluation &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp; &nbsp;|
| ----------- | ----------- | ----------- |
| Cooccurrence      | MultinomialNB       | Accuracy : 45% |
|    |         | Precision : 53%|
|    |         | Recall : 33%|
|    |         | F1-score : 29% |
|    | BernoulliNB       | Accuracy : 48%|
|    |         | Precision :10% |
|    |         | Recall : 20%|
|    |         | F1-score : 13%|
| Cooccurrence & data cleaning      | MultinomialNB       | Accuracy : 48%|
|    |         | Precision : 46%|
|    |         | Recall : 33%|
|    |         | F1-score : 34%|
|    | BernoulliNB       | Accuracy : 48%|
|    |         | Precision : 10%|
|    |         | Recall : 20%|
|    |         | F1-score : 13%|
| Collocation      | MultinomialNB       | Accuracy : 52%|
|    |         | Precision : 39%|
|    |         | Recall : 27%|
|    |         | F1-score : 24%|
|    | BernoulliNB       | Accuracy : 54%|
|    |         | Precision : 46%|
|    |         | Recall : 31%|
|    |         | F1-score : 28%|
| Cooccurrence top 25%:      | MultinomialNB       | Accuracy : 46%|
|    |         | Precision :41% |
|    |         | Recall : 29%|
|    |         | F1-score : 30%|
|    | BernoulliNB       | Accuracy : 48%|
|    |         | Precision :10% |
|    |         | Recall : 20%|
|    |         | F1-score : 13%|
| POS tag Cooccurrence:      | MultinomialNB       | Accuracy : 26%|
|    |         | Precision : 19%|
|    |         | Recall : 17%|
|    |         | F1-score : 18%|
|    | BernoulliNB       | Accuracy :  23%|
|    |         | Precision : 15%|
|    |         | Recall : 14%|
|    |         | F1-score : 13%|


Kali ini, model terbaik adalah Cooccurence dengan data cleaning.

